In [26]:
import napari
import numpy as np
import pandas as pd
import os, re, glob, sys
from skimage import io

In [27]:
pipeline_name = "PC" #python and cellpose
ch1_name = 'NR'
ch2_name = 'CC'
data_path = '/home/exacloud/gscratch/HeiserLab/images/'
data_path = '/Users/dane/Documents/CellTrackingProjects/AU565/images/'
plateID = 'AU02001'
#plateID = sys.argv[1]
well = "A1"
field = "1"
field_num = int(field)
well_index = 1
#well_index = int(sys.argv[2])

p_stack_filename = os.path.join(data_path+plateID,"Analysis",pipeline_name,"intermediate_files",plateID+"_P_"+well+"_"+field+"_reg_stack.tif")
cp_mask_stack_filename = p_stack_filename.replace("_P_", "_R_")
cp_mask_stack_filename = cp_mask_stack_filename.replace("reg_stack.tif","masks_stack.tif")
img_p_stack = io.imread(p_stack_filename)
img_r_stack = io.imread(p_stack_filename.replace("_P_","_R_"))
cp_mask_stack = io.imread(cp_mask_stack_filename)
l1_filename = os.path.join(data_path+plateID,"Analysis",pipeline_name,plateID+"_level_1.csv")
l1 = pd.read_csv(l1_filename)
l1 = l1.query('well == @well & field == @field_num')

In [28]:
tracks = l1[["label", "slice","Nuclei_PC_NR_centroid-0", "Nuclei_PC_NR_centroid-1"]]
tracks = tracks.rename(columns={"slice": "time", "Nuclei_PC_NR_centroid-1": "y", "Nuclei_PC_NR_centroid-0": "x"})
tracks = tracks.sort_values(by=['label', 'time']).to_numpy(dtype = "float_")

In [29]:
graph_filename = os.path.join(data_path,plateID,"Analysis",pipeline_name,"intermediate_files","tracking",
                              well,"field_"+field,"results","tracks.csv")
graph = pd.read_csv(graph_filename)[["label","parent"]]
graph = graph[(graph.parent.isin(l1.label))]
graph = graph.set_index('label').T.to_dict('list')

In [30]:
track_color_features =  l1[["label",'parent', 'length','Cell_PC_CC_mean_intensity_ratio', 'Cell_PC_CC_max_intensity_ratio',
       'Cell_PC_CC_min_intensity_ratio','Nuclei_PC_NR_area', 'lineage']]
track_color_features = track_color_features.rename(columns={"slice": "time"})

vertices = tracks[:, 1:]

In [36]:
viewer = napari.view_image(img_p_stack, name = "phase image")
viewer.add_image(img_r_stack, name = "nuclear image")
viewer.add_image(cp_mask_stack, name = "cp masks image")
viewer.add_points(vertices, size=1, name='points', opacity=0.3)
viewer.add_tracks(tracks, features = track_color_features, graph=graph, name='tracks')

napari.run()

None None []
None None []


In [32]:
cp_mask_stack.shape

(193, 1040, 1408)

In [33]:
l1.shape

(45311, 84)

# 

In [34]:
l1.field

0        1
1        1
2        1
3        1
4        1
        ..
45306    1
45307    1
45308    1
45309    1
45310    1
Name: field, Length: 45311, dtype: int64

In [35]:
int(field)

1